In [35]:
import pandas as pd
from pandas.io.json import json_normalize 
import os
import plotly.express as px
import json 

dataset = "CIFAR100"
all_subdirs = [os.path.join(dataset,d) for d in os.listdir(dataset) if os.path.isdir(os.path.join(dataset, d)) and d != "cifar-100-python"]
print(all_subdirs)
latest_folder = max(all_subdirs, key=os.path.getmtime)

data = []
with open(os.path.join(latest_folder, "results.jsonl"), 'r', encoding='utf-8') as f:
    for line in f:
       data.append(json.loads(line))

df = json_normalize(data)
df = df.applymap(lambda x: x[0] if (isinstance(x, list) and len(x) == 1) else x)

def nice_name(row):
    if row["model"] == "GNCLClassifier":
        if row["l_reg"] == 0:
            return "Ind."
        else:
            return u"GNLC \u03BB = {}".format(row["l_reg"]) 
    elif row["model"] == "BaggingClassifier":
        return "Bagging"
    elif row["model"] == "E2EEnsembleClassifier":
        return "E2E"
    elif row["model"] == "SKLearnModel":
        return "Single"
    elif row["model"] == "SMCLClassifier":
        return "SMCL"
    else:
        return row["model"]

def base_type(row):
    if "binary" in row["base_estimator"]:
        mtype = "binary"
    else:
        mtype = "float"

    if "small" in row["base_estimator"]:
        size = "small"
    else:
        size = "large"

    return row["base_estimator"].split("_")[0] + " " + size + " " + mtype

print(df.columns)     
df["nice_name"] = df.apply(nice_name, axis=1)
df["model_type"] = df.apply(base_type, axis=1)
df["complexity"] = df.apply(lambda x : x["scores.params_test"] if "float" in x["model_type"] else x["scores.params_test"] / 32.0, axis=1)
df["scores.avg_accurcay_test"] = df["scores.avg_accurcay_test"]
df["scores.accuracy_test"] = 100.0*df["scores.accuracy_test"]
df = df.round(decimals = 3)

print("{} experiments done".format(len(df)))
cols = df.columns
cols = ["nice_name", "model_type", "mode", "scores.accuracy_test", "scores.params_test", "scores.diversity_test"] + list(set(cols) - set(["nice_name", "model_type", "mode", "scores.accuracy_test", "scores.params_test", "scores.diversity_test"]))
df = df.reindex(cols, axis=1)
df
#dfbinary = df[df["model_type"] == "binary"]
#dffloat = df[df["model_type"] == "float"]  
#dffloat


['CIFAR100/13-10-2020-22:48:28', 'CIFAR100/13-10-2020-22:33:43', 'CIFAR100/13-10-2020-22:38:46', 'CIFAR100/17-10-2020-22:56:23', 'CIFAR100/17-10-2020-22:37:20', 'CIFAR100/15-10-2020-22:29:04', 'CIFAR100/13-10-2020-22:44:11', 'CIFAR100/13-10-2020-23:46:30', 'CIFAR100/13-10-2020-23:40:44', 'CIFAR100/13-10-2020-23:27:01', 'CIFAR100/13-10-2020-22:40:15', 'CIFAR100/13-10-2020-22:34:54', 'CIFAR100/13-10-2020-23:43:43', 'CIFAR100/15-10-2020-22:21:39', 'CIFAR100/13-10-2020-23:48:31', 'CIFAR100/17-10-2020-22:54:02', 'CIFAR100/13-10-2020-23:48:03', 'CIFAR100/13-10-2020-19:07:51', 'CIFAR100/13-10-2020-22:37:16', 'CIFAR100/13-10-2020-23:49:55', 'CIFAR100/13-10-2020-23:42:29', 'CIFAR100/13-10-2020-23:03:19', 'CIFAR100/15-10-2020-22:18:16', 'CIFAR100/13-10-2020-22:08:02', 'CIFAR100/15-10-2020-22:19:24', 'CIFAR100/15-10-2020-22:04:07', 'CIFAR100/15-10-2020-22:20:11', 'CIFAR100/15-10-2020-22:08:01', 'CIFAR100/15-10-2020-22:06:19', 'CIFAR100/13-10-2020-23:45:49', 'CIFAR100/04-10-2020-20:20:45', 'CIFAR1

,nice_name,model_type,mode,scores.accuracy_test,scores.params_test,scores.diversity_test,scores.avg_accurcay_train,seed,store_model,scoring.loss,...,scores.fit_time,ray_head,optimizer.batch_size,combination_type,scoring.accuracy,train,scoring.avg_loss,scheduler.step_size,no_runs,base_estimator
0,Single,simpleresnet small float,NaN,56.25,88100,0.000,0.000,None,False,loss,...,1505.539,auto,128,NaN,make_scorer(accuracy_score),"[/data/s1/buschjae/CIFAR100/, False]",avg_loss,25,1,"simpleresnet_{'size': 'small', 'model_type': '..."
1,E2E,simpleresnet small float,NaN,67.26,1409600,107.196,3.909,None,False,loss,...,7887.682,auto,128,NaN,make_scorer(accuracy_score),"[/data/s1/buschjae/CIFAR100/, False]",avg_loss,25,1,"simpleresnet_{'size': 'small', 'model_type': '..."
2,SMCLClassifier,simpleresnet small float,NaN,30.31,1409600,5.637,6.246,None,False,loss,...,7941.601,auto,128,best,make_scorer(accuracy_score),"[/data/s1/buschjae/CIFAR100/, False]",avg_loss,25,1,"simpleresnet_{'size': 'small', 'model_type': '..."
3,Bagging,simpleresnet small float,NaN,61.62,1409600,0.522,58.520,None,False,loss,...,7963.127,auto,128,NaN,make_scorer(accuracy_score),"[/data/s1/buschjae/CIFAR100/, False]",avg_loss,25,1,"simpleresnet_{'size': 'small', 'model_type': '..."
4,Ind.,simpleresnet small float,upper,65.32,1409600,0.391,65.197,None,False,loss,...,8239.951,auto,128,average,make_scorer(accuracy_score),"[/data/s1/buschjae/CIFAR100/, False]",avg_loss,25,1,"simpleresnet_{'size': 'small', 'model_type': '..."
5,GNLC λ = 0.1,simpleresnet small float,upper,65.67,1409600,0.437,65.104,None,False,loss,...,8164.890,auto,128,average,make_scorer(accuracy_score),"[/data/s1/buschjae/CIFAR100/, False]",avg_loss,25,1,"simpleresnet_{'size': 'small', 'model_type': '..."
6,GNLC λ = 0.2,simpleresnet small float,upper,65.79,1409600,0.493,64.793,None,False,loss,...,8184.085,auto,128,average,make_scorer(accuracy_score),"[/data/s1/buschjae/CIFAR100/, False]",avg_loss,25,1,"simpleresnet_{'size': 'small', 'model_type': '..."
7,GNLC λ = 0.3,simpleresnet small float,upper,66.50,1409600,0.572,64.493,None,False,loss,...,8196.034,auto,128,average,make_scorer(accuracy_score),"[/data/s1/buschjae/CIFAR100/, False]",avg_loss,25,1,"simpleresnet_{'size': 'small', 'model_type': '..."
8,GNLC λ = 0.4,simpleresnet small float,upper,67.29,1409600,0.678,63.601,None,False,loss,...,8239.779,auto,128,average,make_scorer(accuracy_score),"[/data/s1/buschjae/CIFAR100/, False]",avg_loss,25,1,"simpleresnet_{'size': 'small', 'model_type': '..."
9,GNLC λ = 0.5,simpleresnet small float,upper,67.94,1409600,0.839,62.135,None,False,loss,...,8164.191,auto,128,average,make_scorer(accuracy_score),"[/data/s1/buschjae/CIFAR100/, False]",avg_loss,25,1,"simpleresnet_{'size': 'small', 'model_type': '..."


In [40]:
import numpy as np
import plotly.io as pio

if not os.path.exists("images"):
    os.mkdir("images") 

pio.orca.config.use_xvfb = True

n_estimators = [16]
for mtype in ["simpleresnet small float", "simpleresnet small binary", "simpleresnet large float", "simpleresnet large binary"]:
#mtype, n_estimators = "simpleresnet small float", [16]
#mtype, n_estimators = "simpleresnet small binary", [16]
#mtype, n_estimators = "simpleresnet large float", [16]
#mtype, n_estimators = "simpleresnet large binary", [16]

    methods_order = ["SMCL", "Bagging", "Ind.", "GNLC λ = 0.1", "GNLC λ = 0.2", "GNLC λ = 0.3", "GNLC λ = 0.4" ,"GNLC λ = 0.5", "GNLC λ = 0.6", "GNLC λ = 0.7", "GNLC λ = 0.8", "GNLC λ = 0.9", "E2E"]
    # mtype, n_estimators = "EfficientNetB0 float", [8, 16]
    #mtype, n_estimators = "EfficientNetB0 binary", [8, 16]

    filteredDF = df.loc[ df["model_type"] == mtype ]
    #n_estimators = sorted(set(filteredDF["n_estimators"].dropna()))

    for ne in n_estimators:
        dff = filteredDF.loc[ (filteredDF["n_estimators"] == ne)]
        accuracy_single = filteredDF.loc[filteredDF["n_estimators"].isnull()]["scores.accuracy_test"].to_numpy()[0]
        trainloss_single = filteredDF.loc[filteredDF["n_estimators"].isnull()]["scores.loss_train"].to_numpy()[0]
        #dff = dff.loc[ (dff["l_reg"] <= 0.5) | (dff["l_reg"].isnull())]

        #print(dff)
        #dff = dff.loc[ dff["l_Reg"] < 1000]
        #dff = df[ (df["model_type"] == mtype) and (df["n_estimators"] == ne or df["n_estimators"] == np.nan)]

        fig = px.bar(dff, x="nice_name",y="scores.accuracy_test",
            title="Test accuracy with T = {} and {} models".format(ne, mtype), 
            labels={
                "scores.accuracy_test": "Test accuracy",
                "nice_name": "Method"
            },
            text = "scores.accuracy_test"
        )
        fig.update_traces(textposition='outside')
        fig.update_layout(xaxis={'categoryorder':'array', "categoryarray": methods_order})
        fig.add_shape(type='line',x0=-0.5,y0=accuracy_single,x1=len(methods_order)-1+0.5,
            y1=accuracy_single,line=dict(color='Black',),xref='x',yref='y'
        )
        #fig.update_layout(xaxis={'categoryorder':'category ascending'})
        fig.show()
        fig.write_image("images/{}_{}_{}.pdf".format("Accuracy",mtype.replace(" ", "-"), ne))

        fig = px.bar(dff, x="nice_name",y="scores.diversity_test",
            title="Test diversity with T = {} and {} models".format(ne, mtype), 
            labels={
                "scores.diversity_test": "Diversity",
                "nice_name": "Method"
            },
            log_y = True,
            text = "scores.diversity_test"
        )
        fig.update_traces(textposition='outside')
        #fig.update_layout(xaxis={'categoryorder':'category ascending'})
        fig.update_layout(xaxis={'categoryorder':'array', "categoryarray": methods_order})
        fig.show()
        fig.write_image("images/{}_{}_{}.pdf".format("Diversity",mtype.replace(" ", "-"), ne))

        fig = px.bar(dff, x="nice_name",y="scores.avg_accurcay_test",
            title="Average test accuracy with T = {} and {} models".format(ne, mtype), 
            labels={
                "scores.avg_accurcay_test": "Average test accuracy of each ensemble member",
                "nice_name": "Method"
            },
            text = "scores.avg_accurcay_test"
        )
        fig.update_traces(textposition='outside')
        #fig.update_layout(xaxis={'categoryorder':'category ascending'})
        fig.update_layout(xaxis={'categoryorder':'array', "categoryarray": methods_order})
        fig.show()
        fig.write_image("images/{}_{}_{}.pdf".format("AvgAccuracy",mtype.replace(" ", "-"), ne))

        fig = px.bar(dff, x="nice_name",y="complexity",
            title="Model complexity with T = {} and {} models".format(ne, mtype), 
            labels={
                "complexity": "Model complexity (# of trainable parameters)",
                "nice_name": "Method"
            },
            text = "complexity"
        )
        fig.update_traces(textposition='outside')
        #fig.update_layout(xaxis={'categoryorder':'category ascending'})
        fig.update_layout(xaxis={'categoryorder':'array', "categoryarray": methods_order})
        fig.show()
        fig.write_image("images/{}_{}_{}.pdf".format("Complexity",mtype.replace(" ", "-"), ne))

        fig = px.bar(dff, x="nice_name",y="scores.loss_train",
            title="Training loss with T = {} and {} models".format(ne, mtype), 
            labels={
                "scores.loss_train": "Training loss",
                "nice_name": "Method"
            },
            log_y = True,
            text = "scores.loss_train"
        )
        fig.update_traces(textposition='outside')
        #fig.update_layout(xaxis={'categoryorder':'category ascending'})
        fig.update_layout(xaxis={'categoryorder':'array', "categoryarray": methods_order})
        fig.add_shape(type='line',x0=-0.5,y0=trainloss_single,x1=len(methods_order)-1+0.5,
            y1=trainloss_single,line=dict(color='Black',),xref='x',yref='y'
        )
        fig.show()
        fig.write_image("images/{}_{}_{}.pdf".format("Loss",mtype.replace(" ", "-"), ne))
        

